# Homework 3 - Web Scraping

## 1. Data collection

In [5]:
import bs4
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import requests
import os
from langdetect import detect
import re
import pandas as pd

In [ ]:
# 1.1
# Searching every page and writing the urls of the books in the file.

f = open("List of books (URL).txt", "a", encoding="utf-8")

n_books = []

for k in range(0, 150):
    my_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(k+1)
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()
    soup_i = soup(page_html, "html.parser")
    books = soup_i.findAll("tr")
    n_books.append(len(books))
    for i in range(len(books)):
        item = books[i]
        f.write("https://www.goodreads.com"+item.a["href"]+"\n")
    
f.close()


In [ ]:
# 1.2
# Downloading the html pages of all the books contained in every page.

f = open("List of books (URL).txt", "r")

num = 1

for k in range(0, 150):
    newpath = r"Page "+str(k+1)
    os.makedirs(newpath)
    for i in range(100):
        my_url = f.readline()
        uClient = uReq(my_url)
        page_html = uClient.read()
        uClient.close()
        soup_i = soup(page_html, "html.parser")
        f_new = open(newpath+"/article_"+str(num)+".html", "w", encoding="utf-8")
        f_new.write(str(soup_i))
        f_new.close()
        num += 1

f.close()

In [ ]:
# 1.3
# Acquiring all the information requested of the book pages and creating a tsv file for each of html file.

num = 1
for k in range(0, 150):
    for i in range(100):
        f = open("Page "+str(k+1)+"/article_"+str(num)+".html", "r", encoding="utf-8")
        soup_i = soup(f.read(), "html.parser")
        try:
            Plot = soup_i.find("div", {"id":"description"}).get_text().replace("\n", "").replace("...more", "")
        except:
            Plot = "Nessuna trama"
        if(detect(Plot)!="en" or Plot=="Nessuna trama"):
            f.close()
            num += 1
            continue
        else:
            try:
                bookTitle = soup_i.find("div", {"id":"metacol"}).h1.contents[0].strip("\n ")
            except:
                bookTitle = ""
            try:
                bookSeries = soup_i.find("div", {"id":"metacol"}).h2.a.contents[0].strip("\n ")
            except:
                bookSeries = ""
            try:
                bookAuthors = soup_i.find("a", {"class":"authorName"}).span.contents[0].strip("\n ")
            except:
                bookAuthors = ""
            try:
                ratingValue = soup_i.find("span", {"itemprop":"ratingValue"}).contents[0].strip("\n ")
            except:
                ratingValue = ""
            try:
                ratingCount = soup_i.findAll("a", {"href":"#other_reviews"})[0].get_text().split("\n")[2].strip(" ")
            except:
                ratingCount = ""
            try:
                reviewCount = soup_i.findAll("a", {"href":"#other_reviews"})[1].get_text().split("\n")[2].strip(" ")
            except:
                reviewCount = ""
            try:
                NumberofPages = soup_i.find("span", {"itemprop":"numberOfPages"}).contents[0].split(" ")[0]
            except:
                NumberofPages = ""
            try:
                PublishingDate = soup_i.findAll("div", {"class":"row"})[1].get_text().split("\n")[2].strip(" ")
            except:
                PublishingDate = ""
            
            Url = soup_i.find("link")["href"]
            
            # Different algorithm to retrieve Characters and Setting
            try:
                divs = soup_i.findAll("div", class_="infoBoxRowItem")
                for div in divs:
                    search = div.find(lambda tag: tag.name=="a" and re.match(r"/places/", str(tag.get("href"))))
                    if search:
                        Setting = div
                Setting = Setting.text.strip().replace("\n", " ")
            except:
                Setting = ""
            try:
                ch = soup_i.findAll(lambda tag: tag.name=="a" and len(tag.attrs)==1 and re.match(r"/characters/", str(tag.get("href"))))
                l = []
                for char in ch:
                    l.append(char.text.strip())
                Characters = (" , ".join(l))
            except:
                Characters = ""
            
            f_new = open("Books tsv files/article_"+str(num)+".tsv", "a", encoding="utf-8")
            l = [bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, PublishingDate, Characters, Setting]
            f_new.write(" \t ".join(l))
            f_new.close()
            num += 1
            f.close()

In [28]:
# Example of tsv file considered as a dataframe
pd.read_csv("Books tsv files/article_2.tsv", sep="\t")

,Harry Potter and the Order of the Phoenix,(Harry Potter #5),J.K. Rowling,4.50,"2,531,949","42,890","There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the loomiThere is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.His fate depends on them all.",870,September 2004,"Sirius Black , Draco Malfoy , Ron Weasley , Petunia Dursley , Vernon Dursley , Dudley Dursley , Severus Snape , Rubeus Hagrid , Lord Voldemort , Minerva McGonagall , Neville Longbottom , Fred Weasley , George Weasley , Percy Weasley , Ginny Weasley , Colin Creevey , Filius Flitwick , Gilderoy Lockhart , Lucius Malfoy , Pomona Sprout , Arthur Weasley , Molly Weasley , Cho Chang , Cornelius Fudge , Remus Lupin , Sybil Trelawney , Stan Shunpike , Bellatrix Lestrange , Alastor Moody , Rita Skeeter , Luna Lovegood , Nymphadora Tonks , Dolores Umbridge , Dobby , Kingsley Shacklebolt , Padma Patil , Parvati Patil , Kreacher , Dean Thomas , Seamus Finnigan , Albus Dumbledore , Harry Potter , Hermione Granger , Lavender Brown","Hogwarts School of Witchcraft and Wizardry (United Kingdom) London, England"


## 2. Search Engine

Here it is implemented the search engine for conjunctive (AND) queries, with the plot as the considered attribute for pre-processing and the intersection with the query terms; all the plots are retrieved from their dataframes (tsv files) contained in folder "Books tsv files".
The code blocks below take into consideration plots of the first 500 books and receive in input the query "life journey".
The vocabulary and the indexes can be saved to a text or json file that can be opened before the queries.

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

def pre_process(book_info):
    book_info = re.sub(pattern=" +", repl=" ", string=book_info)
    words = re.sub(pattern="[^\w\s]", repl="", string=book_info)
    words = [token for token in nlp(words) if not token.is_stop]
    words = [token.lemma_ for token in words]
    return [word.lower() for word in words]

attrs = ["bookTitle", "bookSeries", "bookAuthors", "ratingValue", "ratingCount", "reviewCount", "Plot", "NumberofPages", "PublishingDate", "Characters", "Setting"]
 

In [71]:
# Example of a pre_processed plot
pre_process(pd.read_csv("Books tsv files/article_2.tsv", sep="\t",
                         names=attrs)["Plot"][0])

[' ',
 'door',
 'end',
 'silent',
 'corridor',
 'haunting',
 'harry',
 'pottters',
 'dream',
 'wake',
 'middle',
 'night',
 'scream',
 'terrorharry',
 'lot',
 'mind',
 'fifth',
 'year',
 'hogwarts',
 'defense',
 'dark',
 'arts',
 'teacher',
 'personality',
 'like',
 'poison',
 'honey',
 'big',
 'surprise',
 'gryffindor',
 'quidditch',
 'team',
 'loomithere',
 'door',
 'end',
 'silent',
 'corridor',
 'haunting',
 'harry',
 'pottters',
 'dream',
 'wake',
 'middle',
 'night',
 'scream',
 'terrorharry',
 'lot',
 'mind',
 'fifth',
 'year',
 'hogwarts',
 'defense',
 'dark',
 'arts',
 'teacher',
 'personality',
 'like',
 'poison',
 'honey',
 'big',
 'surprise',
 'gryffindor',
 'quidditch',
 'team',
 'loom',
 'terror',
 'ordinary',
 'wizarding',
 'level',
 'exam',
 'thing',
 'pale',
 'grow',
 'threat',
 'hewhomustnotbename',
 ' ',
 'threat',
 'magical',
 'government',
 'authority',
 'hogwart',
 'stopas',
 'grasp',
 'darkness',
 'tighten',
 'harry',
 'discover',
 'true',
 'depth',
 'strength',


In [77]:
# 2.1.1

voc = {}
inv_ind = {}
id = 1
for i in range(0, 500):
    try:
        ds = pd.read_csv("Books tsv files/article_"+str(i+1)+".tsv", sep="\t",
                          names=attrs)
        words = pre_process(ds["Plot"][0].replace(".", ". "))
        for word in words:
            if word not in voc.keys():
                voc[word]=id
                id += 1
        for word in list(set(words)):
            if voc[word] not in inv_ind.keys():
                inv_ind[voc[word]]=[]
                inv_ind[voc[word]].append(i+1)
            else:
                inv_ind[voc[word]].append(i+1)
    except:
        continue

#print(voc, sorted(inv_ind.items()))
    
#with open("vocabulary.txt", "w") as vocabulary:
    #vocabulary.write(str(voc))
#with open("inv_index.txt", "w") as inv_index:
    #inv_index.write(str(inv_ind))

In [88]:
# Example of some words in vocabulary and some lines in inv_ind
count = 0
for key, value in voc.items():
    if count<5 and key!=" ":
        print(key, " : ", value)
    count +=1
count = 0
for key, value in sorted(inv_ind.items()):
    if count<=5 and key!=1:
        print(key, " : ", value
    count +=1

survive  :  2
wild  :  3
sure  :  4
not  :  5
live  :  6
2  :  [1, 32, 45, 54, 56, 70, 86, 151, 168, 176, 187, 209, 214, 223, 230, 310, 320, 321, 325, 330, 375, 381, 388, 391, 402, 403, 407, 442, 455, 477, 485, 486, 499]
3  :  [1, 26, 47, 51, 156, 181, 221, 242, 248, 257, 263, 280, 310, 393, 402, 440, 444, 473]
4  :  [1, 34, 132, 241, 317, 343, 390, 454]
5  :  [1, 5, 18, 22, 31, 37, 42, 44, 71, 84, 90, 92, 100, 103, 123, 132, 139, 140, 155, 165, 168, 173, 185, 187, 201, 202, 210, 213, 214, 218, 223, 224, 227, 239, 240, 241, 244, 269, 270, 284, 287, 306, 311, 317, 319, 321, 337, 339, 342, 343, 345, 350, 352, 356, 359, 369, 370, 382, 384, 390, 391, 393, 396, 402, 406, 418, 432, 435, 439, 443, 445, 446, 451, 452, 458, 477, 487, 488]
6  :  [1, 21, 22, 26, 32, 37, 49, 61, 62, 75, 86, 96, 113, 121, 123, 131, 136, 144, 148, 154, 155, 157, 162, 165, 183, 195, 196, 208, 209, 214, 215, 218, 220, 231, 235, 240, 260, 263, 266, 267, 269, 270, 286, 289, 293, 305, 309, 311, 317, 326, 329, 343, 352, 3

In [13]:
# 2.1.2

import numpy as np
import json

#with open("vocabulary.txt", "r") as vocabulary:
    #voc = json.loads(voc.read())
#with open("inv_index", "r") as inv_index:
    #inv_ind = json.loads(inv_index.read())

# Alternatively (having already the json files for vocabulary and inverted_index):

#with open("vocabulary.json") as vocabulary:
  #voc = json.load(vocabulary)
#with open("inverted_index.json") as inverted_index:
  #inv_ind = json.load(inverted_index)
    
query = input().split(" ")
if len(query)==1:
    if query[0] in voc.keys():
        result = inv_ind[voc[query[0]]]
    else:
        result = []
else:
    l = []
    for i in range(len(query)):
        if query[i] in voc.keys():
            l.append(inv_ind[voc[query[i]]])
    for i in range(len(l)-1):
        inter = list(set.intersection(set(l[i]), set(l[i+1])))
        l[i+1] = inter
    result = l[-1]

data = []
urls = []
for doc_id in result:
    f = open("List of books (URL).txt", "r")
    df = pd.read_csv("Books tsv files/article_"+str(doc_id)+".tsv", sep="\t",
                      names=attrs)
    data.append(df)
    for i in range(doc_id-1):
        f.readline()
    u = f.readline()
    url = pd.DataFrame({"Url":[u]})
    urls.append(url)
    f.close()
dataframe1 = pd.concat(data, axis=0, ignore_index=False)
dataframe2 = pd.concat(urls, axis=0, ignore_index=False)

dataframe = np.hstack([dataframe1, dataframe2])
dataframe = pd.DataFrame(dataframe)
dataframe.iloc[:, [0, 6, 11]]

life journey


,0,6,11
0,Room,"To five-year-old-Jack, Room is the world....T...",https://www.goodreads.com/book/show/31685789-r...
1,The Ultimate Hitchhiker's Guide to the Galaxy,"At last in paperback in one complete volume, ...",https://www.goodreads.com/book/show/13.The_Ult...
2,"Lamb: The Gospel According to Biff, Christ's C...","The birth of Jesus has been well chronicled, ...",https://www.goodreads.com/book/show/28881.Lamb\n
3,"Eat, Pray, Love","A celebrated writer's irresistible, candid, a...",https://www.goodreads.com/book/show/19501.Eat_...
4,City of Heavenly Fire,In this dazzling and long-awaited conclusion ...,https://www.goodreads.com/book/show/8755785-ci...
5,The Subtle Knife,The second instalment in Philip Pullman’s His...,https://www.goodreads.com/book/show/41637836-t...
6,Heart of Darkness,"Heart of Darkness, a novel by Joseph Conrad, ...",https://www.goodreads.com/book/show/4900.Heart...
7,The Odyssey,"Sing to me of the man, Muse, the man of twist...",https://www.goodreads.com/book/show/1381.The_O...
8,"Unbroken: A World War II Story of Survival, Re...","In her long-awaited new book, Laura Hillenbra...",https://www.goodreads.com/book/show/8664353-un...
9,Slaughterhouse-Five,Selected by the Modern Library as one of the ...,https://www.goodreads.com/book/show/4981.Slaug...


In [10]:
import math

def t_freq(term, doc):
    count = 0
    df = pd.read_csv("Books tsv files/article_"+str(doc)+".tsv", sep="\t",
                      names=attrs )
    plot = pre_process(df["Plot"][0])
    for i in range(len(plot)):
        if plot[i]==term:
            count += 1
    return count

def Idf(term):
    N = len(inv_ind[voc[term]])
    files = next(os.walk("Books tsv files"))[2]
    Idf = math.log10(len(files)/N)
    return Idf

def tfIdf(term, doc):
    return t_freq(term, doc)*Idf(term)

def cos_sim(query, doc_id, plot):
    l = []
    inter = list(set.intersection(set(query), set(plot)))
    for q in inter:
        for value in inv_ind2[voc[q]]:
            if value[0]==doc_id:
                l.append(value[1])
    sim = (1/math.sqrt(len(query)))*(1/math.sqrt(len(plot)))*(sum(l))
    return sim

In [ ]:
# 2.2.1

inv_ind2 = {}
for key in voc.keys():
    inv_ind2[voc[key]] = []
    for doc in inv_ind[voc[key]]:
        inv_ind2[voc[key]].append((doc, float("{:.1f}".format(tfIdf(key, doc)))))
#print(inv_ind2)


In [89]:
# Example of some lines in inv_ind2
count = 0
for key, value in sorted(inv_ind2.items()):
    if count<=5 and key!=1:
        print(key, " : ", value)
    count +=1

2  :  [(1, 5.3), (32, 2.6), (45, 2.6), (54, 2.6), (56, 2.6), (70, 5.3), (86, 5.3), (151, 2.6), (168, 2.6), (176, 2.6), (187, 2.6), (209, 2.6), (214, 2.6), (223, 5.3), (230, 5.3), (310, 2.6), (320, 2.6), (321, 5.3), (325, 5.3), (330, 5.3), (375, 5.3), (381, 2.6), (388, 2.6), (391, 5.3), (402, 2.6), (403, 2.6), (407, 5.3), (442, 5.3), (455, 5.3), (477, 2.6), (485, 5.3), (486, 2.6), (499, 5.3)]
3  :  [(1, 5.8), (26, 2.9), (47, 5.8), (51, 8.7), (156, 5.8), (181, 5.8), (221, 2.9), (242, 5.8), (248, 2.9), (257, 5.8), (263, 2.9), (280, 2.9), (310, 2.9), (393, 2.9), (402, 5.8), (440, 2.9), (444, 5.8), (473, 2.9)]
4  :  [(1, 6.5), (34, 6.5), (132, 3.2), (241, 6.5), (317, 3.2), (343, 3.2), (390, 6.5), (454, 6.5)]
5  :  [(1, 4.5), (5, 2.3), (18, 4.5), (22, 2.3), (31, 4.5), (37, 4.5), (42, 4.5), (44, 2.3), (71, 4.5), (84, 4.5), (90, 2.3), (92, 2.3), (100, 4.5), (103, 2.3), (123, 2.3), (132, 2.3), (139, 2.3), (140, 4.5), (155, 4.5), (165, 4.5), (168, 6.8), (173, 4.5), (185, 4.5), (187, 9.0), (201, 

In [15]:
# 2.2.2
import heapq

query = input().split(" ")
if len(query)==1:
    if query[0] in voc.keys():
        result = []
        for j in range(0, len(inv_ind2[voc[query[0]]])):
                result.append(inv_ind2[voc[query[0]]][j][0])
    else:
        result = []
else:
    l1 = []
    for i in range(len(query)):
        l2 = []
        if query[i] in voc.keys():
            for j in range(0, len(inv_ind2[voc[query[i]]])):
                l2.append(inv_ind2[voc[query[i]]][j][0])
        l1.append(l2)
    for i in range(len(l1)-1):
        inter = list(set.intersection(set(l1[i]), set(l1[i+1])))
        l1[i+1] = inter
    result = l1[-1]

data = []
urls = []
Similarities = []
heap = []
for doc_id in result:
    df = pd.read_csv("Books tsv files/article_"+str(doc_id)+".tsv", sep="\t",
                      names=attrs)
    if len(heap)<5:
        heap.append((float("{:.3f}".format(cos_sim(query, doc_id, pre_process(df["Plot"][0])))), doc_id))
    else:
        heapq.heapify(heap)
        heapq.heappushpop(heap, (float("{:.3f}".format(cos_sim(query, doc_id, pre_process(df["Plot"][0])))), doc_id))

for el in heap:
    if el[0]!=0.00:
        df = pd.read_csv("Books tsv files/article_"+str(el[1])+".tsv", sep="\t",
                          names=attrs)
        data.append(df)
        f = open("List of books (URL).txt", "r")
        for i in range(el[1]-1):
            f.readline()
        u = f.readline()
        url = pd.DataFrame({"Url":[u]})
        urls.append(url)
        f.close()
        Similarity = pd.DataFrame({"Similarity":[float("{:.3f}".format(cos_sim(query, el[1], pre_process(df["Plot"][0]))))]})
        Similarities.append(Similarity)
    
dataframe1 = pd.concat(data, axis=0, ignore_index=False)
dataframe2 = pd.concat(urls, axis=0, ignore_index=False)
dataframe3 = pd.concat(Similarities, axis=0, ignore_index=False)

dataframe = np.hstack([dataframe1, dataframe2])
dataframe = np.hstack([dataframe, dataframe3])

# Top-k documents with k=5
top_k = pd.DataFrame(dataframe).iloc[:, [0, 6, 11, 12]]
top_k.sort_values([12], ascending=False)

life journey


,0,6,11,12
3,The Power of One,"In 1939, as Hitler casts his enormous, cruel ...",https://www.goodreads.com/book/show/122.The_Po...,0.838
4,Interview with the Vampire,"This is the story of Louis, as told in his ow...",https://www.goodreads.com/book/show/43763.Inte...,0.721
1,The Subtle Knife,The second instalment in Philip Pullman’s His...,https://www.goodreads.com/book/show/41637836-t...,0.637
2,Mrs. Dalloway,"Heralded as Virginia Woolf's greatest novel, ...",https://www.goodreads.com/book/show/14942.Mrs_...,0.598
0,Slaughterhouse-Five,Selected by the Modern Library as one of the ...,https://www.goodreads.com/book/show/4981.Slaug...,0.546


## 3. Define a new score!

The new scoring function is designed to reward, in the search, the returned books whose plot is more related to the book's title and its setting. For this purpose the function simply counts how many times these two attributes occur in the plot; in addition to this, the frequency of the query in the plot is also taken into consideration.

From this new scoring, fantasy and science-fiction books, for instance, are expected to obtain a higher score since their contents are intrinsically interwined with their fictional world.  

In [16]:
def score_f(query, plot, title, setting):
    count_q = 0
    count_t = 0
    count_s = 0
    
    for q in query:
        for p in plot:
            if p==q:
                count_q += 1
    for t in title:
        for p in plot:
            if p==t:
                count_t += 1
    for s in setting:
        for p in plot:
            if p==s:
                count_s += 1
              
    return (count_q+count_t+count_s)       

In [17]:
query = input().split(" ")
if len(query)==1:
    if query[0] in voc.keys():
        result = inv_ind[voc[query[0]]]
    else:
        result = []
else:
    l = []
    for i in range(len(query)):
        if query[i] in voc.keys():
            l.append(inv_ind[voc[query[i]]])
    for i in range(len(l)-1):
        inter = list(set.intersection(set(l[i]), set(l[i+1])))
        l[i+1] = inter
    result = l[-1]
    
data = []
urls = []
Similarities = []
heap = []
for doc_id in result:
    df = pd.read_csv("Books tsv files/article_"+str(doc_id)+".tsv", sep="\t",
                      names=attrs)
    if len(heap)<5:
        heap.append((score_f(query, pre_process(df["Plot"][0]), pre_process(df["bookTitle"][0]), pre_process(df["Setting"][0])), doc_id))
    else:
        heapq.heapify(heap)
        heapq.heappushpop(heap, (score_f(query, pre_process(df["Plot"][0]), pre_process(df["bookTitle"][0]), pre_process(df["Setting"][0])), doc_id))

for el in heap:
    if el[0]!=0.00:
        df = pd.read_csv("Books tsv files/article_"+str(el[1])+".tsv", sep="\t",
                          names=attrs)
        data.append(df)
        f = open("List of books (URL).txt", "r")
        for i in range(el[1]-1):
            f.readline()
        u = f.readline()
        url = pd.DataFrame({"Url":[u]})
        urls.append(url)
        f.close()
        Similarity = pd.DataFrame({"Similarity":[score_f(query, pre_process(df["Plot"][0]), pre_process(df["bookTitle"][0]), pre_process(df["Setting"][0]))]})
        Similarities.append(Similarity)
    
dataframe1 = pd.concat(data, axis=0, ignore_index=False)
dataframe2 = pd.concat(urls, axis=0, ignore_index=False)
dataframe3 = pd.concat(Similarities, axis=0, ignore_index=False)

dataframe = np.hstack([dataframe1, dataframe2])
dataframe = np.hstack([dataframe, dataframe3])

# Top-k documents with k=5
top_k = pd.DataFrame(dataframe).iloc[:, [0, 6, 11, 12]]
top_k.sort_values([12], ascending=False)

life journey


,0,6,11,12
2,Cold Mountain,Cold Mountain is a novel about a soldier’s pe...,https://www.goodreads.com/book/show/10920.Cold...,24
3,Interview with the Vampire,"This is the story of Louis, as told in his ow...",https://www.goodreads.com/book/show/43763.Inte...,22
4,Mrs. Dalloway,"Heralded as Virginia Woolf's greatest novel, ...",https://www.goodreads.com/book/show/14942.Mrs_...,16
1,"Eat, Pray, Love","A celebrated writer's irresistible, candid, a...",https://www.goodreads.com/book/show/19501.Eat_...,15
0,"Unbroken: A World War II Story of Survival, Re...","In her long-awaited new book, Laura Hillenbra...",https://www.goodreads.com/book/show/8664353-un...,13


## 5. Algorithmic Question

In [25]:
# 1

def seq_len(string, ind):
    if len(string)==1:
        return 1
    l1 = [j for j in range(0, len(string[:ind])) if string[j]<string[ind]]
    if l1==[]:
        return 1
    else:
        l2 = [seq_len(string, j) for j in l1]
        return 1 + max(l2)

def max_len(S):
    l = []
    for i in range(0, len(S)):
        l.append(seq_len(S, i))
    len_max_subseq = max(l)
    return len_max_subseq

In [24]:
# Examples with 3 different strings
S = "CAD"
print(max_len(S))
S = "CADFECE"
print(max_len(S))
S = "CADFECEILGJHABNOPSTIRYOEABILCNR"
print(max_len(S))
S1 = "CADFECEILGJHABNOPSTIRYOEABILCNRASDFHGYTNROEITYMNBCJIURHTMSDFWERTKHMGFUQWERTYGJHKILMNMVDSDFGBVYCDEFGILMNOPTUVWFGBXDRTUNMKJLWSXCFL"

2
3
11


1. The recursive algorithm proves correct and efficient when dealing with short and medium-length strings; however this is not the case for longer strings like $S1$, where the running time does not meet reasonable standards.
2. The algorithm's running time is of course exponential since the function seq_len is called for every index of the string, meanwhile, due to the ricorsion, the same function is called for every $j$ of the string with $S[j]<S[i]$, $i$ being the index where the subsequence terminates. The expression of $T(n)$ can be derived from the one of Fibonacci function (recursive): this time seg_len function is called $f(n-1),...,f(0)$ times instead of two. It was shown in class that, using $T(n)>=2T(n-2)+const$ inequality, $T(n)$ is exponential. 
3.(See below)
4. The new function below computes the result far more efficiently and way faster; the running time considered appears to be $O(n=len(S1))$, as the for loop goes all the way through the length of the string.

In [26]:
# 3

def dynam(S1):
    lst_alph = []
    lst_count = []
    for char in S1:
        if len(lst_alph)==0 or all(np.array(lst_alph)>=char):
            count = 1
        else:
            indexes = np.array(lst_alph)<char
            smaller_counts = np.array(lst_count)[indexes]
            count = max(smaller_counts) + 1
    
        lst_alph.append(char)
        lst_count.append(count)

    print(max(lst_count))

In [31]:
dynam(S1)

19
